In [ ]:
# PAPERMILL Parameters
# points
# crs
# output_file

# Quick Map

In [ ]:
from IPython.display import IFrame
from snippets import *
import pathlib as pl
import geopandas as gpd

### Prep polygon data

In [ ]:
polys_gdf = gpd.read_file(polys)
polys_gdf.crs=crs
polys = quick_transform(polys_gdf, crs)

### Prep polyline data

In [ ]:
lines_gdf = gpd.read_file(lines)
lines_gdf.crs=crs
lines = quick_transform(lines_gdf, crs)

### Prep point data

In [ ]:
points_gdf = gpd.read_file(points)
points_gdf.crs=crs
points = quick_transform(points_gdf, crs)

### Assign Map Center

In [ ]:
xcoord, ycoord = get_centroid(points)
m = interactive_map(ycoord, xcoord)

### Add Geodata to Map

In [ ]:
m = add_poly_popups(polys, m,polys_info,color='#00cc99',fillcolor='#00cc99')
m = add_line_popups(lines, m, lines_info, color='#0099ff')
m = add_point_popups(points, m, points_info,color='#000000',fillcolor='#000000')

### Make popup for points

In [ ]:
m = add_tiles(m)
m = add_measure(m)

### Save to html

In [ ]:
m.save(outfile=output_file)  
IFrame('{}.html'.format(frame), width=800, height=650)

# END